In [ ]:
import intake
import iris
import geoviews as gv
import cartopy.crs as ccrs
import cartopy.feature as cf

from holoviews.operation.datashader import regrid

gv.extension('bokeh')

Guidence on wind shear for aviation
===================================
We're expecting high shear conditions over the UK this week. This analysis will identify risk regions and issue shear warning accordingly.

### Calculate shear in the Mogreps G runs
I'm going to pull in data from the Met Office global ensemble forcast, Mogreps G

In [ ]:
intake.gui

I'm going to calculate shear on pressure levels for Mogreps G.

In [ ]:
from dask_kubernetes import KubeCluster
import distributed
cluster = KubeCluster()
cluster.adapt(minimum=1, maximum=30)
client = distributed.Client(cluster)
cluster

In [ ]:
direction = intake.cat.mo_aws_earth.mogreps_g.wind_from_direction_at_pressure.read()
speed = intake.cat.mo_aws_earth.mogreps_g.wind_speed_at_pressure.read()

In [ ]:
from opscentretools import sheartools
shear = sheartools.calculate_shear(speed, direction)
shear

In [ ]:
from jade_utils.iris_tools import estimate_cube_size
datasize = estimate_cube_size(shear)
datasize

### Exploratory plot of windshear

We need to investigate the windshear field to get a feel for it

First we'll create a plottable object from the data field.

In [ ]:
from opscentretools import plotting
# width = 1000
# height = 600
this_cube = shear[0, 0, 0, ::10]
shear_plot = plotting.interactive_plot(this_cube, coastlines=True)

Next we get the weather warning tool

In [ ]:
from opscentretools import annotable
orange_warning_pen, orange_warning = annotable.warning("orange")

And plot them together

In [ ]:
shear_interactive = shear_plot * orange_warning

### Consult with the chief
I'd like to highlight areas in my advice, but I'd like to check with the cheif to see if they're on the same page.

In [ ]:
annotable.make(shear_interactive.select(pressure=96350), websocket_origin='pangeo-dev.informaticslab.co.uk')

In [ ]:
orange_warning_pen.element.data

### Check the average wind shear profile

In [ ]:
max_wind_shear = shear.collapsed(["forecast_period", "forecast_reference_time", "realization", "latitude", "longitude"], iris.analysis.MAX)
max_wind_shear

### Calculate proportion of MOGREPS-G run that exceeds 15 m/s and 25 m/s shear limits for aviation

In [ ]:
from opscentretools import sheartools
prob_shear_15 = sheartools.calculate_ensemble_exceedence(shear, threshold=15)
prob_shear_25 = sheartools.calculate_ensemble_exceedence(shear, threshold=25)

In [ ]:
from opscentretools import plotting
speed_plot = plotting.interactive_plot(speed[0], coastlines=True)
# speed_plot

In [ ]:
fifteen_interactive = plotting.interactive_plot(prob_shear_15, cmap='summer', coastlines=True)
# fifteen_interactive

In [ ]:
twentyfive_interactive = plotting.interactive_plot(prob_shear_25, cmap='summer', coastlines=True)
# twentyfive_interactive

## Interactive dashboard of plots

We have thresholds for wind shear at `15 m/s` and `25 m/s` want to compare the pressure levels at which these limits are exceeded.

In [ ]:
app = plotting.dashboard_column([fifteen_interactive, twentyfive_interactive], shared_slider=True)
app

In [ ]:
# Show is currently blocked by this browser
app.show(websocket_origin='pangeo-dev.informaticslab.co.uk', port=9999)

## View shear field in 3D

In [ ]:
import itkwidgets as itkw
itkw.view(shear.data[0].copy(), cmap='X Ray', shadow=False)